In [1]:
import pandas as pd 
import numpy as np 
import xlrd 
import sklearn

In [2]:
table_specie = pd.read_excel('../data/raw/LCMS_data_copy.xlsx',0)
table_total= pd.read_excel('../data/raw/LCMS_data_copy.xlsx',1)
lip_name_table = pd.read_excel('../data/raw/LCMS_data_copy.xlsx',4)

In [3]:
lip_name_table

,Original name,Updated name,Class,BioGrouping,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Abreviation,old abbreviation,Lipid class
0,Sph(d16:1) Results,Sph(d16:1),Sph,1,NaN,NaN,NaN,NaN,FFA,FFA,free fatty acids
1,Sph(d18:1) Results,Sph(d18:1),Sph,1,NaN,NaN,NaN,NaN,Cer(d18:0/xx:xx),dhCer,dihydroceramide
2,Sph(d18:2) Results,Sph(d18:2),Sph,1,NaN,NaN,NaN,NaN,Cer(d18:1/xx:x),Cer,ceramide
3,S1P(d16:1) Results,S1P(d16:1),S1P,2,NaN,NaN,NaN,NaN,Hex1Cer,MHC,monohexosylceramide
4,S1P(d18:0) Results,S1P(d18:0),S1P,2,NaN,NaN,NaN,NaN,Hex2Cer,DHC,dihexosylceramide
...,...,...,...,...,...,...,...,...,...,...,...
795,LPC 20:4_OH Results,LPC(20:4) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,LPC 22:6_OH Results,LPC(22:6) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,PC 34:2_OH Results,PC(34:2) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,PC 36:4(a\b)_OH Results,PC(36:4) [+OH],OxSpecies,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
feature_list_total = (table_total.iloc[:,2:].columns.values.tolist())[1:] # id and features name
feature_list_specie = (table_specie.iloc[:,2:].columns.values.tolist())[1:] # id and features name 
name_group_link = lip_name_table[['Updated name','Class','BioGrouping']]

In [7]:
len(feature_list_specie)

774

In [8]:
group_name_list = [name_group_link.loc[name_group_link['Updated name']==item]['Class'].values[0] for item in feature_list_specie]# get group name for each specie

In [12]:
len(group_name_list)

774

In [11]:
len(set(group_name_list)) # nunmber of total groups

41

In [15]:
group_name_dict = {}
for i,item in enumerate(set(group_name_list)):
    group_name_dict[item] = i
group_name_dict

{'PS': 0,
 'TG(O) [NL]': 1,
 'PI': 2,
 'Acylcarnitine': 3,
 'DG': 4,
 'GM1': 5,
 'PE(O)': 6,
 'CE': 7,
 'Sulfatide': 8,
 'OxSpecies': 9,
 'LPI': 10,
 'PC': 11,
 'DE': 12,
 'PE(P)': 13,
 'FFA': 14,
 'LPE(P)': 15,
 'COH': 16,
 'C1P': 17,
 'SM': 18,
 'TG(SIM)': 19,
 'PC(P)': 20,
 'LPE': 21,
 'Cer': 22,
 'TG [NL]': 23,
 'MHC': 24,
 'PA': 25,
 'PE': 26,
 'PC(O)': 27,
 'dhCer': 28,
 'S1P': 29,
 'THC': 30,
 'LPC': 31,
 'DHC': 32,
 'GM3': 33,
 'TG(O)': 34,
 'LPC(P)': 35,
 'PIP1': 36,
 'Ubiquinone': 37,
 'PG': 38,
 'Sph': 39,
 'LPC(O)': 40}

In [9]:
def id_process(id):

    if id[-1] == 'C':
        # pattern 'num + C' 
        return id[:-2]
    elif id[1] == '-':
        # pattern 'P-num' 
        return id[2:]
    else:# pattern 'P num'
        return id[1:]
table_specie.iloc[:,2].apply(id_process)

0      1001
1      1010
2      1019
3      1028
4      1037
       ... 
802    2395
803    2412
804    2429
805    2456
806    2493
Name: Patient ID, Length: 807, dtype: object

In [11]:
# binarize the lip feature: for each feature ,
#  if val > median, set feature_high = 1,feature_low = 0
#  else,   set feature_high = 0,feature_high = 1

table_specie_new = pd.DataFrame()
table_specie_new['Patient_ID'] = table_specie.iloc[:,2].apply(id_process)

for lip_name in feature_list_specie:
    df = table_specie[lip_name]
    # normalized_df = (df-df.min())/(df.max()-df.min())
    # normalized_df = (df-df.mean())/df.std()
    normalized_df = df.fillna(0)

    group_name = name_group_link.loc[name_group_link['Updated name']==lip_name]['Class'].values[0]
    group_name_idx = str(group_name_dict[group_name])
    table_specie_new['lip_' + lip_name + '_' +group_name +'_'+'high'+ '_' + group_name_idx] =  (normalized_df>=normalized_df.median())*1
    table_specie_new['lip_' + lip_name + '_' +group_name +'_'+'low'+ '_' + group_name_idx] = (normalized_df<normalized_df.median())*1 

table_specie_new    

,Patient_ID,lip_Sph(d16:1)_Sph_high_35,lip_Sph(d16:1)_Sph_low_35,lip_Sph(d18:1)_Sph_high_35,lip_Sph(d18:1)_Sph_low_35,lip_Sph(d18:2)_Sph_high_35,lip_Sph(d18:2)_Sph_low_35,lip_S1P(d16:1)_S1P_high_6,lip_S1P(d16:1)_S1P_low_6,lip_S1P(d18:0)_S1P_high_6,...,lip_LPC(18:2) [+OH]_OxSpecies_high_18,lip_LPC(18:2) [+OH]_OxSpecies_low_18,lip_LPC(20:4) [+OH]_OxSpecies_high_18,lip_LPC(20:4) [+OH]_OxSpecies_low_18,lip_LPC(22:6) [+OH]_OxSpecies_high_18,lip_LPC(22:6) [+OH]_OxSpecies_low_18,lip_PC(34:2) [+OH]_OxSpecies_high_18,lip_PC(34:2) [+OH]_OxSpecies_low_18,lip_PC(36:4) [+OH]_OxSpecies_high_18,lip_PC(36:4) [+OH]_OxSpecies_low_18
0,1001,1,0,0,1,0,1,1,0,1,...,0,1,1,0,1,0,0,1,1,0
1,1010,1,0,1,0,1,0,0,1,1,...,1,0,1,0,1,0,1,0,1,0
2,1019,1,0,1,0,0,1,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,1028,0,1,0,1,0,1,0,1,1,...,1,0,1,0,1,0,1,0,1,0
4,1037,0,1,0,1,0,1,1,0,1,...,0,1,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2395,1,0,0,1,0,1,0,1,0,...,1,0,1,0,0,1,1,0,1,0
803,2412,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,1,0,1,0
804,2429,1,0,0,1,0,1,1,0,0,...,1,0,1,0,1,0,1,0,1,0
805,2456,1,0,1,0,0,1,1,0,1,...,0,1,1,0,0,1,0,1,1,0


In [20]:


# table_specie_new_bi = pd.DataFrame()
# table_specie_new_bi['Patient_ID'] = table_specie_new['Patient_ID']
# feature_list = list(table_specie_new)


# for lip_name in feature_list[1:]:
#     df = table_specie_new[lip_name]
#     table_specie_new_bi[lip_name + '_high'] = (df>=df.median())*1
#     table_specie_new_bi[lip_name + '_low'] = (df<df.median())*1


# table_specie_new_bi

,Patient_ID,lip_Sph(d16:1)_high,lip_Sph(d16:1)_low,lip_Sph(d18:1)_high,lip_Sph(d18:1)_low,lip_Sph(d18:2)_high,lip_Sph(d18:2)_low,lip_S1P(d16:1)_high,lip_S1P(d16:1)_low,lip_S1P(d18:0)_high,...,lip_LPC(18:2) [+OH]_high,lip_LPC(18:2) [+OH]_low,lip_LPC(20:4) [+OH]_high,lip_LPC(20:4) [+OH]_low,lip_LPC(22:6) [+OH]_high,lip_LPC(22:6) [+OH]_low,lip_PC(34:2) [+OH]_high,lip_PC(34:2) [+OH]_low,lip_PC(36:4) [+OH]_high,lip_PC(36:4) [+OH]_low
0,1001,1,0,0,1,0,1,1,0,1,...,0,1,1,0,1,0,0,1,1,0
1,1010,1,0,1,0,1,0,0,1,1,...,1,0,1,0,1,0,1,0,1,0
2,1019,1,0,1,0,0,1,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,1028,0,1,0,1,0,1,0,1,1,...,1,0,1,0,1,0,1,0,1,0
4,1037,0,1,0,1,0,1,1,0,1,...,0,1,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2395,1,0,0,1,0,1,0,1,0,...,1,0,1,0,0,1,1,0,1,0
803,2412,1,0,1,0,1,0,1,0,1,...,0,1,0,1,0,1,1,0,1,0
804,2429,1,0,0,1,0,1,1,0,0,...,1,0,1,0,1,0,1,0,1,0
805,2456,1,0,1,0,0,1,1,0,1,...,0,1,1,0,0,1,0,1,1,0


In [12]:
table_specie_new.to_csv('../data/processed/process_lip_specie_binary.csv',index=False)
# table_specie_new_bi.to_csv('../processed_data/process_lip_specie_binary.csv',index=False)

In [13]:
idset = set(table_specie_new['Patient_ID'])
len(idset) # 2two overlaping id 

805

805